In [3]:
import pandas as pd
df = pd.read_csv("https://raw.githubusercontent.com/Laxminarayen/Inceptez-Batch-17/main/Data%20Preprocessing%202/Data/bank-full.csv",sep = ';')

In [24]:
df.shape

(45211, 17)

In [5]:
df_working = df.copy()

In [6]:
df_working.head()

,age,job,marital,education,default,balance,housing,loan,contact,day,month,duration,campaign,pdays,previous,poutcome,y
0,58,management,married,tertiary,no,2143,yes,no,unknown,5,may,261,1,-1,0,unknown,no
1,44,technician,single,secondary,no,29,yes,no,unknown,5,may,151,1,-1,0,unknown,no
2,33,entrepreneur,married,secondary,no,2,yes,yes,unknown,5,may,76,1,-1,0,unknown,no
3,47,blue-collar,married,unknown,no,1506,yes,no,unknown,5,may,92,1,-1,0,unknown,no
4,33,unknown,single,unknown,no,1,no,no,unknown,5,may,198,1,-1,0,unknown,no


In [7]:
df_working['y'].value_counts()

no     39922
yes     5289
Name: y, dtype: int64

In [9]:
df_working.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 45211 entries, 0 to 45210
Data columns (total 17 columns):
 #   Column     Non-Null Count  Dtype 
---  ------     --------------  ----- 
 0   age        45211 non-null  int64 
 1   job        45211 non-null  object
 2   marital    45211 non-null  object
 3   education  45211 non-null  object
 4   default    45211 non-null  object
 5   balance    45211 non-null  int64 
 6   housing    45211 non-null  object
 7   loan       45211 non-null  object
 8   contact    45211 non-null  object
 9   day        45211 non-null  int64 
 10  month      45211 non-null  object
 11  duration   45211 non-null  int64 
 12  campaign   45211 non-null  int64 
 13  pdays      45211 non-null  int64 
 14  previous   45211 non-null  int64 
 15  poutcome   45211 non-null  object
 16  y          45211 non-null  object
dtypes: int64(7), object(10)
memory usage: 5.9+ MB


In [11]:
df_working.describe().T

,count,mean,std,min,25%,50%,75%,max
age,45211.0,40.936210,10.618762,18.0,33.0,39.0,48.0,95.0
balance,45211.0,1362.272058,3044.765829,-8019.0,72.0,448.0,1428.0,102127.0
day,45211.0,15.806419,8.322476,1.0,8.0,16.0,21.0,31.0
duration,45211.0,258.163080,257.527812,0.0,103.0,180.0,319.0,4918.0
campaign,45211.0,2.763841,3.098021,1.0,1.0,2.0,3.0,63.0
pdays,45211.0,40.197828,100.128746,-1.0,-1.0,-1.0,-1.0,871.0
previous,45211.0,0.580323,2.303441,0.0,0.0,0.0,0.0,275.0


In [13]:
#Check for duplicates - Impurity done! 
df_working.duplicated().sum()

0

In [14]:
#Check for null values - Impurity Done! 
df_working.isna().sum()

age          0
job          0
marital      0
education    0
default      0
balance      0
housing      0
loan         0
contact      0
day          0
month        0
duration     0
campaign     0
pdays        0
previous     0
poutcome     0
y            0
dtype: int64

In [18]:
#Check for inconsistencies
import numpy as np
df_working.select_dtypes(exclude = np.number).columns


Index(['job', 'marital', 'education', 'default', 'housing', 'loan', 'contact',
       'month', 'poutcome', 'y'],
      dtype='object')

In [23]:
#Target leakage XXXXXXXXXX
#Feature Engineering 
for i in df_working.select_dtypes(include = ['object']).columns[:-1]:
    print("Column Name: ",i)
    print(df_working[i].value_counts())
    print("-"*100)
    
#Exercise 1: Impute the unknown values with mode for the columns JOB and Education 
#Exercise 2: change the admin. to admin

Column Name:  job
blue-collar      9732
management       9458
technician       7597
admin.           5171
services         4154
retired          2264
self-employed    1579
entrepreneur     1487
unemployed       1303
housemaid        1240
student           938
unknown           288
Name: job, dtype: int64
----------------------------------------------------------------------------------------------------
Column Name:  marital
married     27214
single      12790
divorced     5207
Name: marital, dtype: int64
----------------------------------------------------------------------------------------------------
Column Name:  education
secondary    23202
tertiary     13301
primary       6851
unknown       1857
Name: education, dtype: int64
----------------------------------------------------------------------------------------------------
Column Name:  default
no     44396
yes      815
Name: default, dtype: int64
---------------------------------------------------------------------------------

In [26]:
df_working['job'].replace({'admin.':'admin'},inplace = True)

In [29]:
df_working['job'].value_counts()

blue-collar      9732
management       9458
technician       7597
admin            5171
services         4154
retired          2264
self-employed    1579
entrepreneur     1487
unemployed       1303
housemaid        1240
student           938
Name: job, dtype: int64

In [28]:
df_working['job'].replace({'unknown':np.nan},inplace = True)
df_working['education'].replace({'unknown':np.nan},inplace = True)

In [31]:
df_working['job'].isna().sum()

288

In [32]:
df_working['education'].isna().sum()

1857

In [35]:
df_working['job'].fillna(df_working['job'].mode()[0],inplace = True)

In [34]:
df_working['education'].fillna(df_working['education'].mode()[0],inplace = True)

In [36]:
df_working['job'].isna().sum()

0

In [37]:
df_working['education'].isna().sum()

0

df_working['job']= df_working['job'].replace({'unknown':df_working['education'].mode()[0]})

In [39]:
import numpy as np 
for col_name in df_working.select_dtypes(include = np.number).columns:
    print(col_name)
    q1 = df_working[col_name].quantile(0.25)
    q3 = df_working[col_name].quantile(0.75)
    iqr = q3-q1
    
    low = q1-1.5*iqr
    high = q3+1.5*iqr
    print("Low value:",low)
    print("high value:",high)
    print("Outliers = ",df_working.loc[(df_working[col_name] <low) | (df_working[col_name]>high),col_name])
    df_working.loc[(df_working[col_name] <low) | (df_working[col_name]>high),col_name] = df_working[col_name].median()

age
Low value: 10.5
high value: 70.5
Outliers =  29158    83
29261    75
29263    75
29322    83
29865    75
         ..
45163    71
45191    75
45204    73
45207    71
45208    72
Name: age, Length: 487, dtype: int64
balance
Low value: -1962.0
high value: 3462.0
Outliers =  34       10635
65        6530
69       12223
70        5935
186       4384
         ...  
45164     4655
45181     6879
45185     4256
45191     3810
45208     5715
Name: balance, Length: 4729, dtype: int64
day
Low value: -11.5
high value: 40.5
Outliers =  Series([], Name: day, dtype: int64)
duration
Low value: -221.0
high value: 643.0
Outliers =  37       1666
43       1492
53        787
59       1778
61        812
         ... 
45124     701
45199    1166
45200    1556
45206     977
45208    1127
Name: duration, Length: 3235, dtype: int64
campaign
Low value: -2.0
high value: 6.0
Outliers =  758       7
780       8
906       7
1103      8
1105      7
         ..
44666     7
44680     9
44770     9
44886    12
4505

In [40]:
df_working

,age,job,marital,education,default,balance,housing,loan,contact,day,month,duration,campaign,pdays,previous,poutcome,y
0,58,management,married,tertiary,no,2143,yes,no,unknown,5,may,261,1,-1,0,unknown,no
1,44,technician,single,secondary,no,29,yes,no,unknown,5,may,151,1,-1,0,unknown,no
2,33,entrepreneur,married,secondary,no,2,yes,yes,unknown,5,may,76,1,-1,0,unknown,no
3,47,blue-collar,married,secondary,no,1506,yes,no,unknown,5,may,92,1,-1,0,unknown,no
4,33,blue-collar,single,secondary,no,1,no,no,unknown,5,may,198,1,-1,0,unknown,no
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
45206,51,technician,married,tertiary,no,825,no,no,cellular,17,nov,180,3,-1,0,unknown,yes
45207,39,retired,divorced,primary,no,1729,no,no,cellular,17,nov,456,2,-1,0,unknown,yes
45208,39,retired,married,secondary,no,448,no,no,cellular,17,nov,180,5,-1,0,success,yes
45209,57,blue-collar,married,secondary,no,668,no,no,telephone,17,nov,508,4,-1,0,unknown,no


In [42]:
#Encoding!!!!!!!!!!!!!!!!!! En - code 
#Dummy variable encoding or One hot encoding 

df_working_education = df_working.loc[:,['education','day']]

In [47]:
df_working_education['education'].value_counts()

secondary    25059
tertiary     13301
primary       6851
Name: education, dtype: int64

In [50]:
df_working_numerical = df_working.select_dtypes(include=np.number)
df_working_categorical = df_working.select_dtypes(exclude = np.number).drop(columns = ['y'])

In [55]:
from sklearn.preprocessing import StandardScaler

std_scale = StandardScaler()
df_std = std_scale.fit_transform(df_working_numerical)

In [57]:
numerical_preprocessed_data = pd.DataFrame(df_std,columns = df_working_numerical.columns)

In [58]:
numerical_preprocessed_data

,age,balance,day,duration,campaign,pdays,previous
0,1.760223,1.900256,-1.298476,0.435837,-0.882181,0.0,0.0
1,0.349717,-0.738238,-1.298476,-0.374133,-0.882181,0.0,0.0
2,-0.758537,-0.771936,-1.298476,-0.926386,-0.882181,0.0,0.0
3,0.651969,1.105214,-1.298476,-0.808572,-0.882181,0.0,0.0
4,-0.758537,-0.773185,-1.298476,-0.028055,-0.882181,0.0,0.0
...,...,...,...,...,...,...,...
45206,1.054970,0.255254,0.143418,-0.160596,0.691536,0.0,0.0
45207,-0.154035,1.383541,0.143418,1.871693,-0.095322,0.0,0.0
45208,-0.154035,-0.215282,0.143418,-0.160596,2.265253,0.0,0.0
45209,1.659473,0.059301,0.143418,2.254588,1.478395,0.0,0.0


In [ ]:
KNN - (1.3-2.5-1.2)2 - (y3-y2-y1)2

In [60]:
categorical_preprocessed  =  pd.get_dummies(df_working_categorical)

In [62]:
pd.concat([numerical_preprocessed_data,categorical_preprocessed],axis =1)

,age,balance,day,duration,campaign,pdays,previous,job_admin,job_blue-collar,job_entrepreneur,...,month_jun,month_mar,month_may,month_nov,month_oct,month_sep,poutcome_failure,poutcome_other,poutcome_success,poutcome_unknown
0,1.760223,1.900256,-1.298476,0.435837,-0.882181,0.0,0.0,0,0,0,...,0,0,1,0,0,0,0,0,0,1
1,0.349717,-0.738238,-1.298476,-0.374133,-0.882181,0.0,0.0,0,0,0,...,0,0,1,0,0,0,0,0,0,1
2,-0.758537,-0.771936,-1.298476,-0.926386,-0.882181,0.0,0.0,0,0,1,...,0,0,1,0,0,0,0,0,0,1
3,0.651969,1.105214,-1.298476,-0.808572,-0.882181,0.0,0.0,0,1,0,...,0,0,1,0,0,0,0,0,0,1
4,-0.758537,-0.773185,-1.298476,-0.028055,-0.882181,0.0,0.0,0,1,0,...,0,0,1,0,0,0,0,0,0,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
45206,1.054970,0.255254,0.143418,-0.160596,0.691536,0.0,0.0,0,0,0,...,0,0,0,1,0,0,0,0,0,1
45207,-0.154035,1.383541,0.143418,1.871693,-0.095322,0.0,0.0,0,0,0,...,0,0,0,1,0,0,0,0,0,1
45208,-0.154035,-0.215282,0.143418,-0.160596,2.265253,0.0,0.0,0,0,0,...,0,0,0,1,0,0,0,0,1,0
45209,1.659473,0.059301,0.143418,2.254588,1.478395,0.0,0.0,0,1,0,...,0,0,0,1,0,0,0,0,0,1


In [63]:
pd.get_dummies(df_working_education['education']).head(10)

,primary,secondary,tertiary
0,0,0,1
1,0,1,0
2,0,1,0
3,0,1,0
4,0,1,0
5,0,0,1
6,0,0,1
7,0,0,1
8,1,0,0
9,0,1,0
